In [1]:
import numpy as np
import pandas as pd
import cPickle
from collections import defaultdict
import re

In [30]:
# allData = pd.read_csv("all_data.csv", header = 0)
# allData.dropna(axis = 0, how = 'any', inplace = True)
# allData.reset_index(inplace=True)
# allData.to_csv("all_data_bad_rows_dropped.csv", index=False)

In [57]:
data_train = pd.read_csv("chase_train.csv", header = 0)
data_train.dropna(axis = 0, how = 'any', inplace = True)
data_train.reset_index(inplace=True)
data_train.shape

(52334, 3)

In [58]:
data_test = pd.read_csv("chase_test.csv", header = 0)
data_test.dropna(axis = 0, how = 'any', inplace = True)
data_test.reset_index(inplace=True)
data_test.shape

(52334, 3)

In [59]:
def build_data_train_test(data_train, data_test, train_ratio = 0.8, clean_string=True):
    """
    Loads data and split into train and test sets.
    """
    revs = []
    vocab = defaultdict(float)
    # Pre-process train data set
    for i in xrange(data_train.shape[0]):
        line = data_train['review'][i]
        y = data_train['rating'][i]
        rev = []
        rev.append(line.strip())
        if clean_string:
            orig_rev = clean_str(' '.join(rev))
        else:
            orig_rev = ' '.join(rev).lower()
        words = set(orig_rev.split())
        for word in words:
            vocab[word] += 1
        datum  = {'y': y, 
                  'text': orig_rev,
                  'num_words': len(orig_rev.split()),
                  'split': int(np.random.rand() < train_ratio)}
        revs.append(datum)
        
    # Pre-process test data set
    for i in xrange(data_test.shape[0]):
        line = data_test['review'][i]
        rev = []
        rev.append(line.strip())
        if clean_string:
            orig_rev = clean_str(' '.join(rev))
        else:
            orig_rev = ' '.join(rev).lower()
        words = set(orig_rev.split())
        for word in words:
            vocab[word] += 1
        datum  = {'y': -1, 
                  'text': orig_rev,
                  'num_words': len(orig_rev.split()),
                  'split': -1}
        revs.append(datum)
        
    return revs, vocab

    
def get_W(word_vecs, k=300):
    """
    Get word matrix. W[i] is the vector for word indexed by i
    """
    vocab_size = len(word_vecs)
    word_idx_map = dict()
    W = np.zeros(shape=(vocab_size+1, k), dtype=np.float32)
    W[0] = np.zeros(k, dtype=np.float32)
    i = 1
    for word in word_vecs:
        W[i] = word_vecs[word]
        word_idx_map[word] = i
        i += 1
    return W, word_idx_map

def load_bin_vec(fname, vocab):
    """
    Loads 300x1 word vecs from Google (Mikolov) word2vec
    """
    word_vecs = {}
    with open(fname, 'rb') as f:
        header = f.readline()
        vocab_size, layer1_size = map(int, header.split())
        binary_len = np.dtype('float32').itemsize * layer1_size
        for line in xrange(vocab_size):
            word = []
            while True:
                ch = f.read(1)
                if ch == ' ':
                    word = ''.join(word)
                    break
                if ch != '\n':
                    word.append(ch)   
            if word in vocab:
                word_vecs[word] = np.fromstring(f.read(binary_len), dtype='float32')  
            else:
                f.read(binary_len)
    return word_vecs

def add_unknown_words(word_vecs, vocab, min_df=1, k=300):
    """
    For words that occur in at least min_df documents, create a separate word vector.    
    0.25 is chosen so the unknown vectors have (approximately) same variance as pre-trained ones
    """
    for word in vocab:
        if word not in word_vecs and vocab[word] >= min_df:
            word_vecs[word] = np.random.uniform(-0.25,0.25,k)  

def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
#     string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
#     string = re.sub(r"\'s", " \'s", string) 
#     string = re.sub(r"\'ve", " \'ve", string) 
#     string = re.sub(r"n\'t", " n\'t", string) 
#     string = re.sub(r"\'re", " \'re", string) 
#     string = re.sub(r"\'d", " \'d", string) 
#     string = re.sub(r"\'ll", " \'ll", string) 
#     string = re.sub(r",", " , ", string) 
#     string = re.sub(r"!", " ! ", string) 
#     string = re.sub(r"\(", " \( ", string) 
#     string = re.sub(r"\)", " \) ", string) 
#     string = re.sub(r"\?", " \? ", string) 
#     string = re.sub(r"\s{2,}", " ", string)    
    return string.strip().lower()

In [60]:
w2v_file = 'GoogleNews-vectors-negative300.bin'

revs, vocab = build_data_train_test(data_train, data_test, train_ratio=0.8, clean_string=True)
max_l = np.max(pd.DataFrame(revs)['num_words'])
print 'data loaded!'
print 'number of sentences: ' + str(len(revs))
print 'vocab size: ' + str(len(vocab))
print 'max sentence length: ' + str(max_l)
print 'loading word2vec vectors...',
w2v = load_bin_vec(w2v_file, vocab)
print 'word2vec loaded!'
print 'num words already in word2vec: ' + str(len(w2v))
add_unknown_words(w2v, vocab)
W, word_idx_map = get_W(w2v)
cPickle.dump([revs, W, word_idx_map, vocab], open('chase-train-val-test.pickle', 'wb'))
print 'dataset created!'

data loaded!
number of sentences: 104668
vocab size: 23373
max sentence length: 643
loading word2vec vectors... word2vec loaded!
num words already in word2vec: 14153
dataset created!


In [61]:
from keras import backend
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import Adadelta
from keras.constraints import unitnorm
from keras.regularizers import l2

from sklearn.metrics import roc_auc_score

In [62]:
def get_idx_from_sent(sent, word_idx_map, max_l=51, kernel_size=5):
    """
    Transforms sentence into a list of indices. Pad with zeroes.
    """
    x = []
    pad = kernel_size - 1
    for i in xrange(pad):
        x.append(0)
    words = sent.split()
    for word in words:
        if word in word_idx_map:
            x.append(word_idx_map[word])
    while len(x) < max_l+2*pad:
        x.append(0)
    return x

def make_idx_data(revs, word_idx_map, max_l=51, kernel_size=5):
    """
    Transforms sentences into a 2-d matrix.
    """
    train, val, test = [], [], []
    for rev in revs:
        sent = get_idx_from_sent(rev['text'], word_idx_map, max_l, kernel_size)
        sent.append(rev['y'])
        if rev['split'] == 1:
            train.append(sent)
        elif rev['split'] == 0:
            val.append(sent)
        elif rev['split'] == -1:
            test.append(sent)

    train = np.array(train, dtype=np.int)
    val = np.array(val, dtype=np.int)
    test = np.array(test, dtype=list)

    return [train, val, test]

In [63]:
print "loading data..."
x = cPickle.load(open("chase-train-val-test.pickle", "rb"))
revs, W, word_idx_map, vocab = x[0], x[1], x[2], x[3]
print "data loaded!"
print(len(x[0]), len(x[1]) , len(x[2]), len(x[3]))

loading data...
data loaded!
(104668, 23374, 23373, 23373)


In [64]:
# print(x[0])
datasets = make_idx_data(revs, word_idx_map, max_l=2633, kernel_size=5)

In [104]:
print(datasets[0].shape[0])
print(W.shape[1])
print( int(datasets[0].shape[1]-1))
print(train_Y[1])
print(W.shape[0])

42247
300
2641
[0 0 0 0 0 1]
23374


In [91]:
N = datasets[0].shape[0]
conv_input_width = W.shape[1]
conv_input_height = int(datasets[0].shape[1]-1)

# For each word write a word index (not vector) to X tensor
train_X = np.zeros((N, conv_input_height), dtype=np.int)
train_Y = np.zeros((N, 6), dtype=np.int)
for i in xrange(N):
    for j in xrange(conv_input_height):
        train_X[i, j] = datasets[0][i, j]
    train_Y[i, datasets[0][i, -1]] = 1
    
print 'train_X.shape = {}'.format(train_X.shape)
print 'train_Y.shape = {}'.format(train_Y.shape)

train_X.shape = (42247, 2641)
train_Y.shape = (42247, 6)


In [92]:
Nv = datasets[1].shape[0]

# For each word write a word index (not vector) to X tensor
val_X = np.zeros((Nv, conv_input_height), dtype=np.int)
val_Y = np.zeros((Nv, 6), dtype=np.int)
for i in xrange(Nv):
    for j in xrange(conv_input_height):
        val_X[i, j] = datasets[1][i, j]
    val_Y[i, datasets[1][i, -1]] = 1
    
print 'val_X.shape = {}'.format(val_X.shape)
print 'val_Y.shape = {}'.format(val_Y.shape)

val_X.shape = (10087, 2641)
val_Y.shape = (10087, 6)


In [108]:
backend.set_image_dim_ordering('th')

# Number of feature maps (outputs of convolutional layer)
N_fm = 300
# kernel size of convolutional layer
kernel_size = 8

model = Sequential()
# Embedding layer (lookup table of trainable word vectors)
model.add(Embedding( 
                    output_dim=W.shape[1], 
                    input_length=conv_input_height,
                    weights=[W], 
                    input_dim=W.shape[0],
                    W_constraint=unitnorm()))
# Reshape word vectors from Embedding to tensor format suitable for Convolutional layer
model.add(Reshape((1, conv_input_height, conv_input_width)))

# first convolutional layer
model.add(Convolution2D(N_fm, 
                        kernel_size, 
                        conv_input_width, 
                        border_mode='valid', 
                        W_regularizer=l2(0.0001)))
# ReLU activation
model.add(Activation('relu'))

# aggregate data in every feature map to scalar using MAX operation
model.add(MaxPooling2D(pool_size=(conv_input_height-kernel_size+1, 1)))

model.add(Flatten())
model.add(Dropout(0.5))
# Inner Product layer (as in regular neural network, but without non-linear activation function)
model.add(Dense(6))
# SoftMax activation; actually, Dense+SoftMax works as Multinomial Logistic Regression
model.add(Activation('softmax'))

# Custom optimizers could be used, though right now standard adadelta is employed
opt = Adadelta(lr=1.0, rho=0.95, epsilon=1e-6)
model.compile(loss='categorical_crossentropy', 
              optimizer=opt,
              metrics=['accuracy'])

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:15: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(output_dim=300, weights=[array([[ ..., input_dim=23374, embeddings_constraint=<keras.con..., input_length=2641)`
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(300, (8, 300), padding="valid", kernel_regularizer=<keras.reg...)`


In [109]:
epoch = 0
val_acc = []
val_auc = []

In [110]:
epochs = 3

for i in xrange(epochs):
    model.fit(train_X, train_Y, batch_size=50, nb_epoch=1, verbose=1)
    output = model.predict_proba(val_X, batch_size=10, verbose=1)
    # find validation accuracy using the best threshold value t
    vacc = np.max([np.sum((output[:,1]>t)==(val_Y[:,1]>0.5))*1.0/len(output) for t in np.arange(0.0, 1.0, 0.01)])
    # find validation AUC
    vauc = roc_auc_score(val_Y, output)
    val_acc.append(vacc)
    val_auc.append(vauc)
    print 'Epoch {}: validation accuracy = {:.3%}, validation AUC = {:.3%}'.format(epoch, vacc, vauc)
    epoch += 1
    
print '{} epochs passed'.format(epoch)
print 'Accuracy on validation dataset:'
print val_acc
print 'AUC on validation dataset:'
print val_auc

Epoch 1/1
10087/10087 [==============================] - 729s     

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [ ]:

# Test data preparation
Nt = datasets[2].shape[0]

# For each word write a word index (not vector) to X tensor
test_X = np.zeros((Nt, conv_input_height), dtype=np.int)
for i in xrange(Nt):
    for j in xrange(conv_input_height):
        test_X[i, j] = datasets[2][i, j]
    
print 'test_X.shape = {}'.format(test_X.shape)